In [1]:
import pandas as pd
import pymongo
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# # create a connection to Postgres
# conn = psycopg2.connect(database="postgres", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
# conn.autocommit = True
# cursor = conn.cursor()

# # create the database in Postgres
# create_database_sql = open('../sql/create_db.sql','r')
# cursor.execute(create_database_sql.read())

# #If no error print message
# print("Database created successfully........")

# #Closing the connection and database
# conn.close()
# create_database_sql.close()

In [3]:
# read in csv file from output_data folder
csv_file = "../output_data/tweets.csv"
twitter_df = pd.read_csv(csv_file)
twitter_df.head()

,Unnamed: 0,Tweets
0,0,Tough loss today to a very tough Friends Acade...
1,1,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...
2,2,RT @JamaicaObserver: A St Catherine man is in ...
3,3,RT @arvinjoaquin: A 65-year-old Filipino woman...
4,4,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...


In [4]:
# rename column from Uppercase to Lowercase
twitter_df.rename(columns = {'Tweets':'tweets'}, inplace = True) 
twitter_df.head()

,Unnamed: 0,tweets
0,0,Tough loss today to a very tough Friends Acade...
1,1,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...
2,2,RT @JamaicaObserver: A St Catherine man is in ...
3,3,RT @arvinjoaquin: A 65-year-old Filipino woman...
4,4,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...


In [5]:
# drop the Unnamed column in the df
twitter_df = twitter_df.drop(columns=['Unnamed: 0'])
twitter_df.head()

,tweets
0,Tough loss today to a very tough Friends Acade...
1,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...
2,RT @JamaicaObserver: A St Catherine man is in ...
3,RT @arvinjoaquin: A 65-year-old Filipino woman...
4,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...


In [6]:
# set the index to the 'tweets' column in the df
twitter_df = twitter_df.set_index('tweets')
twitter_df.head()

""
tweets
Tough loss today to a very tough Friends Academy team. Need to bounce back Monday at Malverne! ⚽️🐸
"RT @misanharriman: NEW IMAGES OF RICHARD OKOROGHEYE. In the last confirmed sighting on Monday, 22 March, Richard was seen boarding the 23 b…"
RT @JamaicaObserver: A St Catherine man is in custody after trying to set a woman on fire after attacking her on Monday.\nhttps://t.co/NqpEU…
"RT @arvinjoaquin: A 65-year-old Filipino woman beaten to the ground on her way to church. This incident happened in New York, Monday mornin…"
"RT @misanharriman: NEW IMAGES OF RICHARD OKOROGHEYE. In the last confirmed sighting on Monday, 22 March, Richard was seen boarding the 23 b…"


In [7]:
# create a connection to Postgres
conn = psycopg2.connect(database="twitter_db", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()

In [8]:
# run the schema for the database using the schema file created and the connection to Postgres
schema_sql = open('../sql/schema.sql','r')
cursor.execute(schema_sql.read())

#If no error print message
print("Database Schema Loaded Successfully (Tables Created)........")

# close the schema file
schema_sql.close()

Database Schema Loaded Successfully (Tables Created)........


In [9]:
# create the database connection
connection_string ="postgres:Shjais2014@localhost:5432/twitter_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
# confirm tables
engine.table_names()

['twitter_tweets']

In [11]:
# Load DataFrame into the database
twitter_df.to_sql(name='twitter_tweets', con=engine, if_exists='append', index = True)

In [12]:
# confirm the Select from table in the DataBase
select_sql = open('../sql/select_df.sql','r')
twit_df = pd.read_sql_query(select_sql.read(), con=engine)

#Close File after use
select_sql.close()

In [13]:
# calling in the table data
twit_df

,tweets
0,Tough loss today to a very tough Friends Acade...
1,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...
2,RT @JamaicaObserver: A St Catherine man is in ...
3,RT @arvinjoaquin: A 65-year-old Filipino woman...
4,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...
...,...
195,RT @misanharriman: NEW IMAGES OF RICHARD OKORO...
196,RT @CPRNews: Family and friends gathered today...
197,RT @Kwesiblack__: Good Morning guys\nIs Monday...
198,RT @Myanmar_Now_Eng: Three people were killed ...


In [14]:
#closing the connection to Postgres
conn.close()